<a href="https://colab.research.google.com/github/cjstntjd/project_study/blob/master/movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive',force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [22]:
import numpy as np
import json
import pandas as pd

meta = pd.read_csv('/gdrive/My Drive/스터디/개인과제/영화추천/the-movies-dataset/movies_metadata.csv',low_memory=False)
print(meta.head())
meta = meta[['id', 'original_title', 'original_language', 'genres']]
meta = meta.rename(columns={'id':'movieId'})
meta = meta[meta['original_language'] == 'en']
print(meta.head())
ratings = pd.read_csv('/gdrive/My Drive/스터디/개인과제/영화추천/the-movies-dataset/ratings_small.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
print(ratings.head())
print(ratings.describe())
meta.movieId = pd.to_numeric(meta.movieId, errors='coerce')
ratings.movieId = pd.to_numeric(ratings.movieId, errors='coerce')
def parse_genres(genres_str):
    genres = json.loads(genres_str.replace('\'', '"'))
    
    genres_list = []
    for g in genres:
        genres_list.append(g['name'])

    return genres_list

meta['genres'] = meta['genres'].apply(parse_genres)
print(meta.head())
data = pd.merge(ratings, meta, on='movieId', how='inner')

print(data.head())
matrix = data.pivot_table(index='userId', columns='original_title', values='rating')

print(matrix.head(20))
GENRE_WEIGHT = 0.1

#피어슨 상관 계수 사용 
def pearsonR(s1, s2):
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))

def recommend(input_movie, matrix, n, similar_genre=True):
    input_genres = meta[meta['original_title'] == input_movie]['genres'].iloc(0)[0]

    result = []
    for title in matrix.columns:
        if title == input_movie:
            continue

        # 레이팅 비교
        cor = pearsonR(matrix[input_movie], matrix[title])
        
        # 장르비교 고장 자꾸남 
        if similar_genre and len(input_genres) > 0:
            temp_genres = meta[meta['original_title'] == title]['genres'].iloc(0)[0]

            same_count = np.sum(np.isin(input_genres, temp_genres))
            cor += (GENRE_WEIGHT * same_count)
        
        if np.isnan(cor):
            continue
        else:
            result.append((title, '{:.2f}'.format(cor), temp_genres))
            
    result.sort(key=lambda r: r[1], reverse=True)

    return result[:n]

recommend_result = recommend('Iron Man', matrix, 10, similar_genre=True)

print(pd.DataFrame(recommend_result, columns = ['Title', 'Correlation', 'Genre']))

   adult  ... vote_count
0  False  ...     5415.0
1  False  ...     2413.0
2  False  ...       92.0
3  False  ...       34.0
4  False  ...      173.0

[5 rows x 24 columns]
  movieId  ...                                             genres
0     862  ...  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1    8844  ...  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2   15602  ...  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3   31357  ...  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4   11862  ...                     [{'id': 35, 'name': 'Comedy'}]

[5 rows x 4 columns]
   userId  movieId  rating
0       1       31     2.5
1       1     1029     3.0
2       1     1061     3.0
3       1     1129     2.0
4       1     1172     4.0
              userId        movieId         rating
count  100004.000000  100004.000000  100004.000000
mean      347.011310   12548.664363       3.543608
std       195.163838   26369.198969       1.058064
min         1.000000       1.000000       0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in double_scalars


                                          Title  ...                                              Genre
0        Star Trek VI: The Undiscovered Country  ...     [Science Fiction, Action, Adventure, Thriller]
1  Star Wars: Episode III - Revenge of the Sith  ...               [Science Fiction, Adventure, Action]
2                                Knight of Cups  ...                                   [Romance, Drama]
3                                     Mad Max 2  ...     [Adventure, Action, Thriller, Science Fiction]
4                    Rambo: First Blood Part II  ...                 [Action, Adventure, Thriller, War]
5                                      Sneakers  ...                             [Comedy, Crime, Drama]
6                            Shaft's Big Score!  ...                [Action, Adventure, Crime, Mystery]
7                  Man with the Screaming Brain  ...                  [Comedy, Horror, Science Fiction]
8                                         Metro  ...       [Acti